## 1. Import Libraries và Khởi tạo Spark Session

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import (
    col, explode, split, trim, current_timestamp, row_number, 
    when, regexp_replace, monotonically_increasing_id, dense_rank
)
from pyspark.sql.window import Window
import os

# Set AWS environment variables for MinIO
os.environ['AWS_REGION'] = 'us-east-1'
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'admin123'

# Khởi tạo Spark Session
spark = (
    SparkSession.builder.appName("Load_Data_To_Gold_Tables")
    .master("spark://spark-master:7077")
    .config("spark.executor.memory", "1536m")
    .config("spark.executor.cores", "2")
    # ===== Iceberg Catalog qua Nessie =====
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v2")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://gold/")
    .config("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    # ===== Cấu hình MinIO =====
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key-id", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-access-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .config("spark.sql.catalog.nessie.s3.region", "us-east-1")
    # ===== Spark + Hadoop S3 connector =====
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.region", "us-east-1")
    .config("spark.executorEnv.AWS_REGION", "us-east-1")
    .config("spark.executorEnv.AWS_ACCESS_KEY_ID", "admin")
    .config("spark.executorEnv.AWS_SECRET_ACCESS_KEY", "admin123")
    .config("spark.jars", "/opt/spark/jars/hadoop-aws-3.3.4.jar,/opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")
print("Spark Session đã được khởi tạo!")
print(f"Spark Master: {spark.sparkContext.master}")
print(f"Application ID: {spark.sparkContext.applicationId}")

## 2. Đọc Dữ Liệu từ Result Model và Lọc Records Chưa Xử Lý

In [ ]:
# Đọc dữ liệu từ bảng result_multi_model
df_result = spark.table("nessie.silver_tables.result_multi_model")

print(f"Tổng số records trong result_multi_model: {df_result.count()}")

# Kiểm tra xem Post table đã có dữ liệu chưa
try:
    df_existing_posts = spark.table("nessie.gold_result_model_multi_task.Post")
    existing_post_ids = [row.postID for row in df_existing_posts.select("postID").distinct().collect()]
    print(f"Tìm thấy {len(existing_post_ids)} posts đã được xử lý trước đó")
    
    # Lọc ra các posts chưa được xử lý
    if existing_post_ids:
        df_result_new = df_result.filter(~col("postID").isin(existing_post_ids))
        print(f"Số lượng posts mới cần xử lý: {df_result_new.count()}")
        
        if df_result_new.count() == 0:
            print("\n Không có posts mới để xử lý. Tất cả dữ liệu đã được load vào Gold tables.")
            print("Nếu muốn load lại, hãy xóa dữ liệu trong Gold tables trước.")
        else:
            df_result = df_result_new
    else:
        print("Không có posts nào trong Gold tables, sẽ xử lý toàn bộ dữ liệu")
        
except Exception as e:
    print(f"Gold tables chưa có dữ liệu hoặc chưa tồn tại: {e}")
    print("Sẽ xử lý toàn bộ dữ liệu từ result_multi_model")

print("\nSchema:")
df_result.printSchema()
print("\nSample data:")
df_result.show(5, truncate=False)

## 3. Transform và Load vào Table Entity

Parse Label_NER để tách các entity (B-MAJOR, I-MAJOR, B-ORG, I-ORG, etc.)

In [ ]:
# Parse NER labels để extract entities
# Label_NER có format: "O O B-MAJOR I-MAJOR O B-ORG I-ORG O"
# description_Normalized có format: "word1 word2 word3 ..."

from pyspark.sql.functions import udf, collect_list

@udf(returnType=ArrayType(StructType([
    StructField("entityType", StringType()),
    StructField("entityName", StringType())
])))
def extract_entities(ner_labels, text):
    """Extract entities from NER labels and text"""
    if not ner_labels or not text:
        return []
    
    labels = ner_labels.split()
    words = text.split()
    
    if len(labels) != len(words):
        return []
    
    entities = []
    current_entity = None
    current_type = None
    current_words = []
    
    for label, word in zip(labels, words):
        if label.startswith('B-'):
            # Lưu entity trước (nếu có)
            if current_entity and current_words:
                entities.append({
                    'entityType': current_type,
                    'entityName': ' '.join(current_words)
                })
            
            # Bắt đầu entity mới
            current_type = label[2:]  # Bỏ 'B-'
            current_words = [word]
            current_entity = True
            
        elif label.startswith('I-') and current_entity:
            # Tiếp tục entity hiện tại
            if label[2:] == current_type:
                current_words.append(word)
        else:
            # Kết thúc entity
            if current_entity and current_words:
                entities.append({
                    'entityType': current_type,
                    'entityName': ' '.join(current_words)
                })
            current_entity = None
            current_type = None
            current_words = []
    
    # Lưu entity cuối cùng
    if current_entity and current_words:
        entities.append({
            'entityType': current_type,
            'entityName': ' '.join(current_words)
        })
    
    return entities

# Extract entities từ tất cả posts
df_entities_raw = df_result.select(
    col("postID"),
    explode(extract_entities(col("Label_NER"), col("description_Normalized"))).alias("entity")
).select(
    col("postID"),
    col("entity.entityType").alias("entityType"),
    col("entity.entityName").alias("entityName")
).filter(
    (col("entityType").isNotNull()) & (col("entityName").isNotNull())
)

print(f"\nTổng số entities được extract: {df_entities_raw.count()}")
df_entities_raw.show(10, truncate=False)

# Lấy danh sách unique entities và gán ID
df_unique_entities = df_entities_raw.select(
    col("entityType"),
    col("entityName")
).distinct()

window_spec = Window.orderBy("entityType", "entityName")
df_entities = df_unique_entities.withColumn(
    "entityID",
    row_number().over(window_spec)
).withColumn(
    "created_at", current_timestamp()
).withColumn(
    "updated_at", current_timestamp()
).select(
    col("entityID"),
    col("entityType"),
    col("entityName"),
    col("created_at"),
    col("updated_at")
)

print(f"\nSố lượng unique entities từ dữ liệu mới: {df_entities.count()}")
print("\nDistribution by entity type:")
df_entities.groupBy("entityType").count().orderBy(col("count").desc()).show()

# Kiểm tra và merge với entities đã có
print("\n=== Loading data into Entity table ===")
try:
    df_existing_entities = spark.table("nessie.gold_result_model_multi_task.Entity")
    existing_count = df_existing_entities.count()
    print(f"Tìm thấy {existing_count} entities đã tồn tại")
    
    # Lấy entityID max để tiếp tục đánh số
    max_entity_id = df_existing_entities.agg({"entityID": "max"}).collect()[0][0]
    if max_entity_id is None:
        max_entity_id = 0
    print(f"Max entityID hiện tại: {max_entity_id}")
    
    # Lọc ra các entities chưa tồn tại
    df_new_entities = df_entities.join(
        df_existing_entities.select("entityType", "entityName"),
        on=["entityType", "entityName"],
        how="left_anti"
    )
    
    new_count = df_new_entities.count()
    print(f"Số lượng entities mới cần thêm: {new_count}")
    
    if new_count > 0:
        # Gán lại entityID bắt đầu từ max_entity_id + 1
        window_spec = Window.orderBy("entityType", "entityName")
        df_new_entities = df_new_entities.withColumn(
            "entityID",
            row_number().over(window_spec) + max_entity_id
        ).select(
            col("entityID"),
            col("entityType"),
            col("entityName"),
            col("created_at"),
            col("updated_at")
        )
        
        # Append vào table
        df_new_entities.writeTo("nessie.gold_result_model_multi_task.Entity") \
            .using("iceberg") \
            .append()
        print(f"✓ Đã thêm {new_count} entities mới!")
        
        # Update df_entities để sử dụng cho các bước sau (bao gồm cả old và new)
        df_entities = df_existing_entities.union(df_new_entities)
    else:
        print("⚠️ Không có entities mới cần thêm")
        df_entities = df_existing_entities
        
except Exception as e:
    print(f"Table Entity chưa có dữ liệu hoặc chưa tồn tại: {e}")
    print("Tạo mới table với dữ liệu hiện tại")
    df_entities.writeTo("nessie.gold_result_model_multi_task.Entity") \
        .using("iceberg") \
        .create()
    print(f"✓ Đã tạo table Entity với {df_entities.count()} records!")

## 4. Transform và Load vào Table Topic

In [ ]:
# Parse Label_Topic (có thể là multi-label, phân cách bằng '|')
# Ví dụ: "Topic1|Topic2" hoặc "Topic1"

df_topics_raw = df_result.select(
    explode(split(col("Label_Topic"), "\\|")).alias("topicName")
).filter(
    (col("topicName").isNotNull()) & 
    (col("topicName") != "None") &
    (trim(col("topicName")) != "")
)

# Lấy danh sách unique topics và gán ID
df_unique_topics = df_topics_raw.select(
    trim(col("topicName")).alias("topicName")
).distinct()

window_spec = Window.orderBy("topicName")
df_topics = df_unique_topics.withColumn(
    "topicID",
    row_number().over(window_spec)
).withColumn(
    "created_at", current_timestamp()
).withColumn(
    "updated_at", current_timestamp()
).select(
    col("topicID"),
    col("topicName"),
    col("created_at"),
    col("updated_at")
)

print(f"Số lượng unique topics từ dữ liệu mới: {df_topics.count()}")
df_topics.show(20, truncate=False)

# Kiểm tra và merge với topics đã có
print("\n=== Loading data into Topic table ===")
try:
    df_existing_topics = spark.table("nessie.gold_result_model_multi_task.Topic")
    existing_count = df_existing_topics.count()
    print(f"Tìm thấy {existing_count} topics đã tồn tại")
    
    # Lấy topicID max để tiếp tục đánh số
    max_topic_id = df_existing_topics.agg({"topicID": "max"}).collect()[0][0]
    if max_topic_id is None:
        max_topic_id = 0
    print(f"Max topicID hiện tại: {max_topic_id}")
    
    # Lọc ra các topics chưa tồn tại
    df_new_topics = df_topics.join(
        df_existing_topics.select("topicName"),
        on="topicName",
        how="left_anti"
    )
    
    new_count = df_new_topics.count()
    print(f"Số lượng topics mới cần thêm: {new_count}")
    
    if new_count > 0:
        # Gán lại topicID bắt đầu từ max_topic_id + 1
        window_spec = Window.orderBy("topicName")
        df_new_topics = df_new_topics.withColumn(
            "topicID",
            row_number().over(window_spec) + max_topic_id
        ).select(
            col("topicID"),
            col("topicName"),
            col("created_at"),
            col("updated_at")
        )
        
        # Append vào table
        df_new_topics.writeTo("nessie.gold_result_model_multi_task.Topic") \
            .using("iceberg") \
            .append()
        print(f"✓ Đã thêm {new_count} topics mới!")
        
        # Update df_topics để sử dụng cho các bước sau
        df_topics = df_existing_topics.union(df_new_topics)
    else:
        print("⚠️ Không có topics mới cần thêm")
        df_topics = df_existing_topics
        
except Exception as e:
    print(f"Table Topic chưa có dữ liệu hoặc chưa tồn tại: {e}")
    print("Tạo mới table với dữ liệu hiện tại")
    df_topics.writeTo("nessie.gold_result_model_multi_task.Topic") \
        .using("iceberg") \
        .create()
    print(f"✓ Đã tạo table Topic với {df_topics.count()} records!")

## 5. Transform và Load vào Table Post

In [ ]:
# Tạo table Post từ result model
df_posts = df_result.select(
    col("postID"),
    col("description_Normalized").alias("description"),
    col("timePublish"),
    col("likeCount"),
    col("commentCount"),
    col("shareCount"),
    col("Label_Intent").alias("intent"),
    col("type")
).withColumn(
    "created_at", current_timestamp()
).withColumn(
    "updated_at", current_timestamp()
)

print(f"Tổng số posts mới: {df_posts.count()}")
df_posts.show(5, truncate=80)

# Load vào table Post (chỉ append posts mới, đã lọc ở bước 2)
print("\n=== Loading data into Post table ===")
if df_posts.count() > 0:
    try:
        df_posts.writeTo("nessie.gold_result_model_multi_task.Post") \
            .using("iceberg") \
            .append()
        print(f"✓ Đã thêm {df_posts.count()} posts mới!")
    except Exception as e:
        if "table does not exist" in str(e).lower() or "not found" in str(e).lower():
            print("Table Post chưa tồn tại, tạo mới...")
            df_posts.writeTo("nessie.gold_result_model_multi_task.Post") \
                .using("iceberg") \
                .create()
            print(f"✓ Đã tạo table Post với {df_posts.count()} records!")
        else:
            raise e
else:
    print("⚠️ Không có posts mới cần thêm")

## 6. Transform và Load vào Table Post_Entity

Bảng quan hệ M:N giữa Post và Entity

In [ ]:
# Extract entities với postID và order
df_post_entities_raw = df_result.select(
    col("postID"),
    explode(extract_entities(col("Label_NER"), col("description_Normalized"))).alias("entity")
).select(
    col("postID"),
    col("entity.entityType").alias("entityType"),
    col("entity.entityName").alias("entityName")
).filter(
    (col("entityType").isNotNull()) & (col("entityName").isNotNull())
)

# Add entity order (thứ tự xuất hiện trong post)
window_spec = Window.partitionBy("postID").orderBy("entityType", "entityName")
df_post_entities_ordered = df_post_entities_raw.withColumn(
    "entityOrder",
    row_number().over(window_spec)
)

# Join với Entity table để lấy entityID
df_post_entity = df_post_entities_ordered.join(
    df_entities.select("entityID", "entityType", "entityName"),
    on=["entityType", "entityName"],
    how="inner"
).select(
    col("postID"),
    col("entityID"),
    col("entityOrder"),
    col("entityName")
).withColumn(
    "created_at", current_timestamp()
).withColumn(
    "updated_at", current_timestamp()
)

print(f"Tổng số quan hệ Post-Entity mới: {df_post_entity.count()}")
df_post_entity.show(10, truncate=False)

# Load vào table Post_Entity (chỉ thêm quan hệ mới)
print("\n=== Loading data into Post_Entity table ===")
if df_post_entity.count() > 0:
    try:
        df_post_entity.writeTo("nessie.gold_result_model_multi_task.Post_Entity") \
            .using("iceberg") \
            .append()
        print(f"✓ Đã thêm {df_post_entity.count()} quan hệ Post-Entity mới!")
    except Exception as e:
        if "table does not exist" in str(e).lower() or "not found" in str(e).lower():
            print("Table Post_Entity chưa tồn tại, tạo mới...")
            df_post_entity.writeTo("nessie.gold_result_model_multi_task.Post_Entity") \
                .using("iceberg") \
                .create()
            print(f"✓ Đã tạo table Post_Entity với {df_post_entity.count()} records!")
        else:
            raise e
else:
    print("⚠️ Không có quan hệ Post-Entity mới cần thêm")

## 7. Transform và Load vào Table Post_Topic

Bảng quan hệ M:N giữa Post và Topic

In [ ]:
# Parse topics cho mỗi post
df_post_topics_raw = df_result.select(
    col("postID"),
    explode(split(col("Label_Topic"), "\\|")).alias("topicName")
).filter(
    (col("topicName").isNotNull()) & 
    (col("topicName") != "None") &
    (trim(col("topicName")) != "")
).select(
    col("postID"),
    trim(col("topicName")).alias("topicName")
)

# Join với Topic table để lấy topicID
df_post_topic = df_post_topics_raw.join(
    df_topics.select("topicID", "topicName"),
    on="topicName",
    how="inner"
).select(
    col("postID"),
    col("topicID")
).withColumn(
    "created_at", current_timestamp()
).withColumn(
    "updated_at", current_timestamp()
)

print(f"Tổng số quan hệ Post-Topic mới: {df_post_topic.count()}")
df_post_topic.show(10, truncate=False)

# Load vào table Post_Topic (chỉ thêm quan hệ mới)
print("\n=== Loading data into Post_Topic table ===")
if df_post_topic.count() > 0:
    try:
        df_post_topic.writeTo("nessie.gold_result_model_multi_task.Post_Topic") \
            .using("iceberg") \
            .append()
        print(f"✓ Đã thêm {df_post_topic.count()} quan hệ Post-Topic mới!")
    except Exception as e:
        if "table does not exist" in str(e).lower() or "not found" in str(e).lower():
            print("Table Post_Topic chưa tồn tại, tạo mới...")
            df_post_topic.writeTo("nessie.gold_result_model_multi_task.Post_Topic") \
                .using("iceberg") \
                .create()
            print(f"✓ Đã tạo table Post_Topic với {df_post_topic.count()} records!")
        else:
            raise e
else:
    print("⚠️ Không có quan hệ Post-Topic mới cần thêm")

## 8. Verify Dữ Liệu Đã Load

In [ ]:
print("="*80)
print("SUMMARY - Data Loaded to Gold Tables")
print("="*80)

tables = [
    "Entity",
    "Topic",
    "Post",
    "Post_Entity",
    "Post_Topic"
]

for table_name in tables:
    df_verify = spark.table(f"nessie.gold_result_model_multi_task.{table_name}")
    count = df_verify.count()
    print(f"\n{'='*80}")
    print(f"Table: {table_name}")
    print(f"Total records: {count}")
    print(f"{'='*80}")
    df_verify.show(5, truncate=False)

print("\n" + "="*80)
print("✓ Hoàn thành load dữ liệu vào tất cả Gold tables!")
print("="*80)

## 9. Thống Kê và Phân Tích

In [ ]:
# Thống kê số lượng entities theo type
print("\n=== Entity Distribution by Type ===")
spark.sql("""
    SELECT entityType, COUNT(*) as count
    FROM nessie.gold_result_model_multi_task.Entity
    GROUP BY entityType
    ORDER BY count DESC
""").show(truncate=False)

# Thống kê số lượng topics
print("\n=== Topic Distribution ===")
spark.sql("""
    SELECT topicName, COUNT(pt.postID) as post_count
    FROM nessie.gold_result_model_multi_task.Topic t
    LEFT JOIN nessie.gold_result_model_multi_task.Post_Topic pt ON t.topicID = pt.topicID
    GROUP BY topicName
    ORDER BY post_count DESC
""").show(20, truncate=False)

# Thống kê intent distribution
print("\n=== Intent Distribution ===")
spark.sql("""
    SELECT intent, COUNT(*) as count
    FROM nessie.gold_result_model_multi_task.Post
    GROUP BY intent
    ORDER BY count DESC
""").show(truncate=False)

# Top entities được nhắc đến nhiều nhất
print("\n=== Top 10 Most Mentioned Entities ===")
spark.sql("""
    SELECT e.entityName, e.entityType, COUNT(pe.postID) as mention_count
    FROM nessie.gold_result_model_multi_task.Entity e
    JOIN nessie.gold_result_model_multi_task.Post_Entity pe ON e.entityID = pe.entityID
    GROUP BY e.entityName, e.entityType
    ORDER BY mention_count DESC
    LIMIT 10
""").show(truncate=False)

## 10. Dừng Spark Session

In [ ]:
# Dừng Spark Session
spark.stop()
print("Spark Session đã được dừng!")